In [ ]:
%pylab inline
from PyGMO import *
import emg
import activation_signal
import muscle
import muscle_utils
import numpy as np
import scipy
import matplotlib.pyplot as plt

In [ ]:

class test_parest(problem.base):
    
    def __init__(self, 
                 muscle_type = muscle_utils.MUSCLE_NAME.BICEPS_BRACHII, 
                 joint = muscle_utils.MUSCLE_JOINT.ELBOW, 
                 emg_pod = emg.EMGPOD.BICEPS_BRACHII, 
                 C1=-0.033, C2=-0.019, A=-0.200, d=0.05, 
                 dim = 9, fdim = 1, cdim = 0):
        super(test_parest, self).__init__(dim, 0, fdim, cdim)
                
        self.muscle_type = muscle_type
        self.joint = joint
        self.emg_pod = emg_pod
        self.C1 = C1
        self.C2 = C2
        self.A = A
        self.d = d
        
        self.emg = emg.EMG()
        
        
        self.emg_measurements = emg_in
        self.angles = angles_in
        self.y = force_out
        
        self.set_bounds(
            [200, 100, 100, 500, 0.5, 1, 1, 0.1, 0.1], 
            [1200, 400, 400, 10000, 1, 10, 10, 10, 10]
        )
        
    
    def simulate(self, x):
        act_sig = activation_signal.ActivationSignal(self.C1, self.C2, self.A, self.d, self.emg_pod)
        self.emg.register_observer(act_sig)
        
        m = muscle.Muscle(
            activation_signal = act_sig,
            max_length = x[0],
            optimal_fiber_length = x[1],
            tensor_slack_length  = x[2],
            max_force = x[3],
            alpha = x[4],
            Spe = x[5],
            Sse = x[6],
            phi_m = x[7],
            phi_v = x[8],
            ts = 0.01
        )
        
        force, actvation = [], []
        
        for i in range(len(self.emg_measurements)):
            e = self.emg_measurements[i]
            a = [self.angles[i], 0]
            self.emg.on_emg_measurement(e)
            actvation.append(act_sig.get_activation_level())
            force.append(m.get_torque_estimate(a, self.joint))
        
        self.emg.unregister_observer(act_sig)
        
        return force, actvation
        
    def _objfun_impl(self, x):
        est, _ = self.simulate(x)
        
        est = np.array(est)
        mse = sum((self.y[50:] - est[50:])**2)
                
        return (mse,)
        
    #def _compute_constraints_impl(self, x):
    #    return (0,)
    
    def __copy__(self):
        return self
        

In [ ]:
import os
import pwd
import muscle_utils
user = pwd.getpwuid( os.getuid() )[0]

kinkom=np.genfromtxt(
    "/home/{}/Dropbox/exo-arm/logs/kincom_logs/morten/MRF1_clean".format(
        user
    )
    , delimiter=',')

angle=abs(kinkom[100:-20,1])
force = kinkom[100:-20,-1]

emg_meas=np.genfromtxt(
    "/home/{}/Dropbox/exo-arm/logs/emg_logs/mr_morten/mr_morten_flex_isokinetic_3rep/emg-1485951230.csv".format(
        user
    ), 
    delimiter=',')

emg_cleaned = emg_meas[300:-300,1:]

emg_resampled = scipy.signal.resample(emg_cleaned, len(angle))


In [ ]:
#angle[test][547:], force[test][547:]
angles_in = angle[test][520:]
emg_in = emg_resampled[test][520:]
force_out = force[test][520:]

In [ ]:
prob = test_parest(
    muscle_type = muscle_utils.MUSCLE_NAME.BICEPS_BRACHII,
    joint = muscle_utils.MUSCLE_JOINT.ELBOW,
    emg_pod = emg.EMGPOD.BICEPS_BRACHII,
    C1=-0.033, C2=-0.019, A=-0.200, d=0.05,
    dim = 9, 
    fdim = 1, 
    cdim = 0
)

In [ ]:
algo = algorithm.pso(gen=100)  # 500 generations of bee_colony algorithm
isl = island(algo, prob, 10)  # Instantiate population with 20 individuals
isl.evolve(1)  # Evolve the island once
isl.join()
print(isl.population.champion)
best = list(isl.population.champion.x)

In [ ]:
t, a = prob.simulate(best)

plt.plot(force_out)
plt.plot(force[test][520:])

figure()
plt.plot(angles_in[50:], t[50:], '*')
plt.plot(angles_in[50:], force_out[50:], '*')
plt.legend(['Simulated', 'Measuremed'])
plt.xlabel('Angle')
plt.ylabel('Torque')

In [ ]:
len(angle)

In [ ]:
test = [i for i in range(1,len(angle)) if angle[i] - angle[i-1] > 0 and angle[i] > 5]

In [ ]:
plt.plot(angle[test][520:], force[test][520:], 'x')
plt.figure()
plt.plot(angle[test][520:], force[test][250:], 'x')

In [ ]:
for i in range(len(angle[test])):
    if angle[i] < 10:
        print(i)